## Week 3 Graded Assignment :- Segmenting and Clustering Neighborhood in Toronto

# PROBLEM 2

In [40]:
# Import the "requests" library to fetch the page content and bs4 ( BeautifulSoup) for Parsing the HTML page content 
import pandas as pd
from bs4 import BeautifulSoup
import requests 



In [41]:
# Scrape a table from Wikipedia using BeautifulSoup, Request,as well as pandas Module. 
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# Make a GET request to fetch the raw HTML content
HTML_Content = requests.get(url).text

# Parse the HTML Content 
soup = BeautifulSoup(HTML_Content, 'xml')
table=soup.find('table')


In [42]:
# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
column_names = ['Postalcode','Borough','Neighborhood']
DataFrame = pd.DataFrame(columns = column_names)

In [43]:
# Search all the postcode, Borough, and Neighborhood in Toronto City
for tr_cell in table.find_all('tr'):
    Input_Data=[]
    for td_cell in tr_cell.find_all('td'):
        Input_Data.append(td_cell.text.strip())
    if len(Input_Data)==3:
        DataFrame.loc[len(DataFrame)] = Input_Data

In [44]:
DataFrame.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [45]:
# Use the .shape method to print the number of rows of in original dataframe.
DataFrame.shape

(180, 3)

## Data Cleaning 

In [46]:
# Process the cells that have an assigned borough in the dataframe. Ignore cells with a borough that is Not assigned.
DataFrame=DataFrame[DataFrame['Borough']!='Not assigned']
DataFrame.head(5)

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [47]:
DataConditioning_df=DataFrame.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
DataConditioning_df=DataConditioning_df.reset_index(drop=False)
DataConditioning_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [48]:
DataFrame_merge = pd.merge(DataFrame, DataConditioning_df, on='Postalcode')

In [49]:
DataFrame_merge.drop(['Neighborhood'],axis=1,inplace=True)

In [50]:
DataFrame_merge.drop_duplicates(inplace=True)

In [51]:
DataFrame_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)

In [52]:
DataFrame_merge.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [53]:
# In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
DataFrame_merge.shape

(103, 3)

In [54]:
pip install geocoder   ## Geocoder was not installed. Therefore, first i have to installed Geocoder Module. 

Note: you may need to restart the kernel to use updated packages.


In [55]:
# Geocoder Python Package 
import geocoder 

def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude


In [56]:
# here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data
import pandas as pd 
geo_df=pd.read_csv('http://cocl.us/Geospatial_data')

In [57]:
geo_df.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [58]:
geo_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
geo_merged = pd.merge(geo_df, DataFrame_merge, on='Postalcode')

In [59]:
geo_data=geo_merged[['Postalcode','Borough','Neighborhood','Latitude','Longitude']]

KeyError: "['Neighbourhood'] not in index"

In [ ]:
geo_data.head()